# Load env and files

In [1]:
import os
os.chdir('..')

In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
from fossil_classification import *
from enrich_holdings import *
from reports_etl import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

In [4]:
response_path = "data/downloaded reports/company reports 2021Q3/"
all_holdings_cls_path = response_path + "all_holdings_cls.csv"
all_holdings_cls = pd.read_csv(all_holdings_cls_path, dtype=holdings_dtypes())

tlv_s2i = prepare_tlv_sec_num_to_issuer(fetch_latest_tlv_sec_num_to_issuer())
isin2lei = fetch_latest_isin2lei()

/Users/urimarom/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,7,24,25,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# adding "I_" to ParentCorpLegalId to avoid confusion with il_corp_num
all_holdings_cls["ParentCorpLegalId"] = "I_" + all_holdings_cls["ParentCorpLegalId"]

In [6]:
# enrich holdings file - fix IDs
all_holdings_cls = add_all_id_types_to_holdings(all_holdings_cls, tlv_s2i, isin2lei)

/Users/urimarom/anaconda3/lib/python3.6/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)



Holding file ISIN col is: מספר ני"ע
number of ISINs: 92388 out of 585803 rows

Holding file מספר תאגיד col is: מספר מנפיק
number of מספר תאגידs: 193701 out of 585803 rows

no LEIs in holdings file
מספר ני"עs with matching ISIN: 202955 out of total relevant rows: 491038
מספר תאגידs with matching מספר מנפיק: 185754 out of total relevant rows: 193701
מספר ני"עs with matching מספר מנפיק: 187639 out of total relevant rows: 491038
ISINs with matching מספר מנפיק: 188220 out of total relevant rows: 202955
ISINs with matching LEI: 48922 out of total relevant rows: 202955


# Let the Analysis begin!

In [7]:
pd.DataFrame(all_holdings_cls.groupby(['SystemName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
SystemName,
ביטוח,"2,632,863,710.94"
גמל,"3,890,353,769.96"
פנסיה,"3,622,905,980.61"


In [74]:
def sum_and_fossil_sum(holdings, group):
    """get sum and fossil sum for holdings by group
    
    """
    sums = pd.DataFrame(holdings.groupby(group, dropna=False).agg(
        {'שווי':'sum', 'שווי פוסילי':'sum'})).reset_index()
    return sums


def get_summary(arg1, *args, holdings=all_holdings_cls):
    '''get summary stats grouped by 1 or more columns, e.g. Company, holding_type
    '''
    group = [arg1]+[*args]
    summary = sum_and_fossil_sum(holdings,group)
    summary["שיעור פוסילי מסך הנכסים"] = 1.00 * summary["שווי פוסילי"] / summary["שווי"]
    
    # add summary of stocks and bonds only
    holdings_cls_trd_stocks_bonds = holdings[holdings["holding_type"].isin(['מניות', 'אג"ח קונצרני'])]
    summary_tradable_stocks_bonds_only = sum_and_fossil_sum(holdings_cls_trd_stocks_bonds,group)
    summary_tradable_stocks_bonds_only["שיעור פוסילי במניות ואגח קונצרני סחירים"] = 1.00 * summary_tradable_stocks_bonds_only[
        "שווי פוסילי"] / summary_tradable_stocks_bonds_only["שווי"]
    summary = summary.merge(summary_tradable_stocks_bonds_only,
                            left_on=group,
                            right_on=group,
                            how='left',
                            suffixes = ['', ' במניות ואגח קונצרני סחירים']
                           )
    
    # add summary of non fossils
    holdings_cls_non_fossil_types = holdings[holdings["holding_type"].isin(get_non_fossil_holding_types())]
    summary_non_fossil_types = pd.DataFrame(
        holdings_cls_non_fossil_types.groupby(group, dropna=False).agg(
            {'שווי':'sum'})
    ).reset_index()
    summary = summary.merge(summary_non_fossil_types,
                            left_on=group,
                            right_on=group,
                            how='left',
                            suffixes = ['', ' בסוגי החזקות לא פוסיליים']
                           )
    summary["שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים"] = 1.00 * summary["שווי פוסילי במניות ואגח קונצרני סחירים"] / (
        summary["שווי במניות ואגח קונצרני סחירים"] + summary["שווי בסוגי החזקות לא פוסיליים"])
    return summary

## Get Company level stats over time

### Filter results to include only major companies

In [82]:
institutions = [
    'מנורה',
    'מגדל',
    'הראל',
    'מיטב דש',
    'אלטשולר',
    'מור',
    'הפניקס',
    'כלל',
    'פסגות',
    'הלמן',
    'אנליסט',
    'ילין'
]

mask = all_holdings_cls["ParentCorpName"].str.startswith(tuple(institutions))

# removing הלמן-אלדובי חח"י גמל בע"מ 515447035 & 520027715, מנורה מבטחים והסתדרות המהנדסים ניהול קופות גמל בע"מ
all_holdings_cls_filtered = all_holdings_cls[mask][~all_holdings_cls[mask]["ParentCorpLegalId"].isin(['I_520027715', 'I_515447035'])]
# all_holdings_cls_filtered.groupby(["ParentCorpName","ParentCorpLegalId"]).size()

all_holdings_cls_filtered['ParentCorpGroup'] = all_holdings_cls_filtered['ParentCorpName'].str.split().str[0].str.split("-").str[0]
all_holdings_cls_filtered['ReportPeriodDate'] = all_holdings_cls_filtered['ReportPeriodDesc'].map(report_period_desc_to_date)
company_stats_since_2020 = get_summary('ReportPeriodDate', 'ParentCorpGroup', holdings=all_holdings_cls_filtered)
company_stats_since_2020 = company_stats_since_2020.sort_values(
    ["ReportPeriodDate", "ParentCorpGroup"], ascending=(False, True)
)

In [83]:
company_stats_since_2020.head()

,ReportPeriodDate,ParentCorpGroup,שווי,שווי פוסילי,שיעור פוסילי מסך הנכסים,שווי במניות ואגח קונצרני סחירים,שווי פוסילי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים,שווי בסוגי החזקות לא פוסיליים,שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים
72,2021-09-30,אלטשולר,"194,508,347.67","2,998,981.25",0.02,"60,159,804.96","2,867,047.70",0.05,"94,619,759.76",0.02
73,2021-09-30,אנליסט,"13,167,973.51","755,870.19",0.06,"4,931,564.02","755,870.19",0.15,"5,223,731.57",0.07
74,2021-09-30,הלמן,"15,677,553.29","720,315.64",0.05,"5,496,495.91","716,727.19",0.13,"5,705,238.74",0.06
75,2021-09-30,הפניקס,"160,887,964.03","7,322,297.17",0.05,"58,382,002.14","7,259,527.67",0.12,"52,370,747.11",0.07
76,2021-09-30,הראל,"208,846,066.56","5,770,384.85",0.03,"52,498,622.43","5,768,266.08",0.11,"70,043,049.77",0.05


In [84]:
# company_stats_since_2020[company_stats_since_2020['ParentCorpGroup'] == 'מגדל']

In [85]:
company_stats_since_2020.to_csv(response_path+"company_stats_since_2020.csv", index=False)

In [86]:
company_system_stats_since_2020 = get_summary(
    'ReportPeriodDate', 'ParentCorpGroup', 'SystemName',
    holdings=all_holdings_cls_filtered
)

In [87]:
company_system_stats_since_2020.to_csv(response_path+"company_system_stats_since_2020.csv", index=False)

## Look into non fossil holding types

In [65]:
all_holdings_cls_non_fossil_types = all_holdings_cls_filtered[
    all_holdings_cls_filtered["holding_type"].isin(get_non_fossil_holding_types())
]

all_holdings_cls_non_fossil_types['holding_type'].unique()

array(['מזומנים', 'תעודות התחייבות ממשלתיות',
       'לא סחיר - תעודות התחייבות ממשלתי', 'פקדונות מעל 3 חודשים'],
      dtype=object)

In [66]:
all_holdings_cls_non_fossil_types.groupby("holding_type").agg({
      "שווי": "sum",
    "שווי פוסילי": "sum"
})

,שווי,שווי פוסילי
holding_type,,
לא סחיר - תעודות התחייבות ממשלתי,"1,040,434,526.80",0.00
מזומנים,"738,522,129.92",0.00
פקדונות מעל 3 חודשים,"53,212,857.20",0.00
תעודות התחייבות ממשלתיות,"1,491,058,879.74","29,677.35"


In [49]:
# holdings that should be non fossil by holding type but are fossil
all_holdings_cls_non_fossil_types[
    all_holdings_cls_non_fossil_types["is_fossil"] == 1
]

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI,ParentCorpGroup,ReportPeriodDate
15844,ISRELE 4.25 14/8/28,None,NaN,Baa2,Moodys,דולר אמריקאי,0.04,0.03,"14,919.64",0.00,0.00,2049749,תעודות התחייבות ממשלתיות,NYSE,NaN,6.34,"4,000,000.00",111.87,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"מור קופות גמל בע""מ",I_514956465,0.0,nan,nan,27/05/2021,2021 רבעון 1,1.00,"14,919.64",IL0060002446,None,NaN,מור,2021-03-31
24555,ETP 5 1/4 04/15,None,NaN,NR,NaN,דולר אמריקאי,0.05,0.08,"2,366.65",0.01,0.00,1847126,תעודות התחייבות ממשלתיות,אחר,NaN,6.89,"576,000.00",115.25,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,08/07/2020,2020 רבעון 1,1.00,"2,366.65",US29278NAG88,None,5493008K8W3OKZE54J59,פסגות,2020-03-31
24559,PEMEX 0 03/11/2,None,NaN,NR,NaN,דולר אמריקאי,0.06,0.02,517.37,0.00,0.00,1847126,תעודות התחייבות ממשלתיות,אחר,NaN,0.19,"176,000.00",82.46,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,08/07/2020,2020 רבעון 1,1.00,517.37,US71654QCF72,None,NaN,פסגות,2020-03-31
123705,ETP 5 1/4 04/15,None,NaN,NR,NaN,דולר אמריקאי,0.05,0.04,"2,183.39",0.01,0.00,1945468,תעודות התחייבות ממשלתיות,אחר,NaN,7.60,"576,000.00",110.16,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,30/11/2020,2020 רבעון 3,1.00,"2,183.39",US29278NAG88,None,5493008K8W3OKZE54J59,פסגות,2020-09-30
123708,PEMEX 0 03/11/2,None,NaN,NR,NaN,דולר אמריקאי,0.06,0.05,600.23,0.00,0.00,1945468,תעודות התחייבות ממשלתיות,אחר,NaN,0.20,"176,000.00",99.11,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,30/11/2020,2020 רבעון 3,1.00,600.23,US71654QCF72,None,NaN,פסגות,2020-09-30
160894,ETP 5 1/4 04/15,None,NaN,NR,NaN,דולר אמריקאי,NaN,NaN,"2,204.06",0.01,0.00,1884615,תעודות התחייבות ממשלתיות,אחר,NaN,nan,"576,000.00",110.40,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,07/09/2020,2020 רבעון 2,1.00,"2,204.06",US29278NAG88,None,5493008K8W3OKZE54J59,פסגות,2020-06-30
160898,PEMEX 0 03/11/2,None,NaN,NR,NaN,דולר אמריקאי,NaN,NaN,592.68,0.00,0.00,1884615,תעודות התחייבות ממשלתיות,אחר,NaN,nan,"176,000.00",97.16,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,07/09/2020,2020 רבעון 2,1.00,592.68,US71654QCF72,None,NaN,פסגות,2020-06-30
174930,ETP 5 1/4 04/15,None,NaN,NR,NaN,דולר אמריקאי,NaN,0.00,"2,331.49",0.01,0.00,2062624,תעודות התחייבות ממשלתיות,אחר,NaN,0.00,"599,000.00",116.75,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,20/06/2021,2021 רבעון 1,1.00,"2,331.49",US29278NAG88,None,5493008K8W3OKZE54J59,פסגות,2021-03-31
174933,PEMEX 0 03/11/2,None,NaN,NR,NaN,דולר אמריקאי,NaN,0.00,859.55,0.00,0.00,2062624,תעודות התחייבות ממשלתיות,אחר,NaN,0.00,"254,000.00",101.50,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,20/06/2021,2021 רבעון 1,1.00,859.55,US71654QCF72,None,NaN,פסגות,2021-03-31
477141,ETP 5 1/4 04/15,None,NaN,NR,NaN,דולר אמריקאי,NaN,NaN,"2,273.40",0.01,0.00,2018568,תעודות התחייבות ממשלתיות,אחר,NaN,0.00,"599,000.00",118.05,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,05/04/2021,2020 רבעון 4,1.00,"2,273.40",US29278NAG88,None,5493008K8W3OKZE54J59,פסגות,2020-12-31


## Export holdings with missing classification that should be classified

In [16]:
missing_cls = all_holdings_cls[
    (all_holdings_cls["holding_type"].isin(['מניות', 'אג"ח קונצרני'])) &
    (all_holdings_cls["is_fossil"].isnull())
]

cols = ['שם המנפיק/שם נייר ערך' ,'מספר ני"ע' ,'מספר מנפיק', 'ISIN', 'מספר תאגיד', 'LEI']
missing_cls_deduped = missing_cls[cols].drop_duplicates()
print(missing_cls_deduped.shape[0])
missing_cls_deduped

13


,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,ISIN,מספר תאגיד,LEI
33299,CHKP US Equity- צ'ק פוינט,IL0010824113 EQUITY,NaN,None,520042821,NaN
43826,PLURISTEM Therapeutics Inc SYS- PLURISTEM THER...,US72940P1066-71013122,NaN,None,None,NaN
60336,PEMEX 5.95 28/01/31,USP71654QDE98,NaN,None,None,NaN
65698,GLAXOSMITHKLINE PLC- GLAXOSMITHKLINE PLC,US37733W1053-110637592,NaN,None,None,NaN
114513,Tencent holdings- Tencent holdings ltd,KY6875721220-60175411,NaN,None,None,NaN
137899,איי.אפ.אפ- אינטרנשיונל פליוורס אנד פראגרנסס אינק,US4595061015_1155019,1760,None,None,NaN
146015,Capital One- capital one,US14040H1059-60004140,NaN,None,None,NaN
168984,P-ADR,70422225,NaN,None,None,NaN
218873,LVS VEGAS SANDS CORP- LAS VEGAS SANDS CORP,71032965 - US5178341070,NaN,None,None,NaN
346953,CYBR 0 11/15/2024,US23248VAA35 C,NaN,None,None,NaN


In [15]:
missing_cls_deduped.to_csv(response_path+"missing_cls.csv", index=False)

# Compare Fossil Holdings over quarters

In [80]:
quarters_for_comparison = ['2021 רבעון 2', '2021 רבעון 3']
company_for_comparison = ['מגדל']

majors_fossil_holdings_for_comparison = all_holdings_cls_filtered[
    (all_holdings_cls_filtered["holding_type"].isin(['מניות', 'אג"ח קונצרני'])) &
    (all_holdings_cls_filtered["is_fossil"] == 1) &
    (all_holdings_cls_filtered["ReportPeriodDesc"].isin(quarters_for_comparison)) &
    (all_holdings_cls_filtered["ParentCorpGroup"].isin(company_for_comparison))
]

majors_fossil_holdings_for_comparison["שווי פוסילי"].sum()

12357277.25933839

In [81]:
majors_fossil_holdings_for_comparison[
    (majors_fossil_holdings_for_comparison["מספר מנפיק"].isnull()) &
    (majors_fossil_holdings_for_comparison["LEI"].isnull())
].head()

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI,clean_name,ParentCorpGroup,ReportPeriodDate
79609,DELEK & AVNER TAMAR 5.082 2023,None,NaN,BBB-,S&P,דולר אמריקאי,0.05,0.05,"63,559.73",0.00,0.00,2101872,"אג""ח קונצרני",אחר,NaN,2.37,"19,368,608.03",100.66,nan,0.06,בלומברג,ENERGY,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"מגדל חברה לביטוח בע""מ",I_520004896,0.0,nan,nan,30/08/2021,2021 רבעון 2,1.00,"63,559.73",IL0011321747,514914001,NaN,DELEK AVNER TAMAR,מגדל,2021-06-30
79610,DELEK & AVNER TAMAR 5.412 2025,None,NaN,BBB-,S&P,דולר אמריקאי,0.05,0.05,"88,218.32",0.00,0.00,2101872,"אג""ח קונצרני",אחר,NaN,4.05,"26,914,421.10",100.54,nan,0.08,בלומברג,ENERGY,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"מגדל חברה לביטוח בע""מ",I_520004896,0.0,nan,nan,30/08/2021,2021 רבעון 2,1.00,"88,218.32",IL0011321820,514914001,NaN,DELEK AVNER TAMAR,מגדל,2021-06-30
79638,ENI 3.375 PERP,None,NaN,Baa3,Moodys,אירו,0.03,0.02,"85,445.92",0.00,0.00,2101872,"אג""ח קונצרני",אחר,NaN,7.04,"20,314,201.41",108.55,nan,0.01,בלומברג,ENERGY,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"מגדל חברה לביטוח בע""מ",I_520004896,0.0,nan,nan,30/08/2021,2021 רבעון 2,1.00,"85,445.92",XS2242931603,None,NaN,ENI,מגדל,2021-06-30
79639,ENIIM 2.75 PERP,None,NaN,Baa3,Moodys,אירו,0.03,0.03,"27,194.08",0.00,0.00,2101872,"אג""ח קונצרני",אחר,NaN,7.96,"6,956,918.29",100.88,nan,0.01,בלומברג,ENERGY,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"מגדל חברה לביטוח בע""מ",I_520004896,0.0,nan,nan,30/08/2021,2021 רבעון 2,1.00,"27,194.08",XS2334857138,None,NaN,ENIIM,מגדל,2021-06-30
79651,SSE SSELN 4.75 9/77 06/22,None,NaN,BBB-,FITCH,דולר אמריקאי,0.05,0.02,"95,787.40",0.00,0.00,2101872,"אג""ח קונצרני",אחר,NaN,1.17,"28,033,597.94",104.81,nan,0.03,בלומברג,UTILITIES,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"מגדל חברה לביטוח בע""מ",I_520004896,0.0,nan,nan,30/08/2021,2021 רבעון 2,1.00,"95,787.40",XS1572343744,None,NaN,SSE,מגדל,2021-06-30


## group holdings for summary stats per held company

### Handle Israeli holdings (having Israeli security num)

In [82]:
# 1. group by Israeli security number
il_fossil_holdings = majors_fossil_holdings_for_comparison.groupby(["ParentCorpGroup", "ReportPeriodDate", 'מספר ני"ע'])
il_fossil_holdings_agg = il_fossil_holdings.agg(
    name=pd.NamedAgg(column="שם המנפיק/שם נייר ערך", aggfunc="first"),
    issuer_num=pd.NamedAgg(column="מספר מנפיק", aggfunc="first"),
    il_corp_num=pd.NamedAgg(column="מספר תאגיד", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="שווי פוסילי", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="ערך נקוב", aggfunc="sum")
).reset_index()

In [83]:
# use il_corp_num when issuer_num when missing
il_fossil_holdings_agg["issuer_num"] = il_fossil_holdings_agg["issuer_num"].fillna(il_fossil_holdings_agg["il_corp_num"])

In [84]:
# 2. group by Israeli issuer number
il_fossil_holdings_by_issuer = il_fossil_holdings_agg.groupby(["ParentCorpGroup", "ReportPeriodDate", 'issuer_num'])
il_fossil_holdings_by_issuer_agg = il_fossil_holdings_by_issuer.agg(
    name=pd.NamedAgg(column="name", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="fossil_sum", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="quantity_sum", aggfunc="sum")
).reset_index()

In [85]:
il_fossil_holdings_by_issuer_agg["fossil_sum"].sum()

8829172.260896308

In [86]:
majors_fossil_holdings_for_comparison[majors_fossil_holdings_for_comparison['מספר ני"ע'].notnull()]["שווי פוסילי"].sum()

8829172.260896308

Great Success!

### Handle non-Israeli holdings

In [87]:
non_il_fossil_holdings = majors_fossil_holdings_for_comparison[majors_fossil_holdings_for_comparison['מספר ני"ע'].isnull()]
non_il_fossil_holdings["שווי פוסילי"].sum()

3528104.9984420817

In [88]:
non_il_fossil_holdings["ISIN"].isnull().sum()

0

In [89]:
# 1. group by ISIN
non_il_fossil_holdings = non_il_fossil_holdings.groupby([
    "ParentCorpGroup", "ReportPeriodDate", 'ISIN'
])
non_il_fossil_holdings_agg = non_il_fossil_holdings.agg(
    name=pd.NamedAgg(column="שם המנפיק/שם נייר ערך", aggfunc="first"),
    issuer_num=pd.NamedAgg(column="מספר מנפיק", aggfunc="first"),
    il_corp_num=pd.NamedAgg(column="מספר תאגיד", aggfunc="first"),
    lei=pd.NamedAgg(column="LEI", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="שווי פוסילי", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="ערך נקוב", aggfunc="sum")
).reset_index()

print(non_il_fossil_holdings_agg["fossil_sum"].sum())
# Fill in issuer_num for missing LEIs, then il_corp_num
non_il_fossil_holdings_agg["issuer_num"] = non_il_fossil_holdings_agg["issuer_num"].fillna(
    non_il_fossil_holdings_agg["lei"])
non_il_fossil_holdings_agg["issuer_num"] = non_il_fossil_holdings_agg["issuer_num"].fillna(
    non_il_fossil_holdings_agg["il_corp_num"])
non_il_fossil_holdings_agg.head()

3528104.998442082


,ParentCorpGroup,ReportPeriodDate,ISIN,name,issuer_num,il_corp_num,lei,fossil_sum,quantity_sum
0,מגדל,2021-06-30,CA29250N1050,ENBRIDGE INC- ENBRIDGE,None,None,None,14.36,110.00
1,מגדל,2021-06-30,FR0011401728,EDF 6 PREP 01/26,549300X3UK4GG3FNMO06,None,549300X3UK4GG3FNMO06,"122,250.19","23,700,000.01"
2,מגדל,2021-06-30,FR0011697028,Electricite De Franc 5 01/26,549300X3UK4GG3FNMO06,None,549300X3UK4GG3FNMO06,"44,318.53","10,000,000.00"
3,מגדל,2021-06-30,FR0013534336,Electricite De France 3.375,549300X3UK4GG3FNMO06,None,549300X3UK4GG3FNMO06,"41,455.83","10,000,000.00"
4,מגדל,2021-06-30,GB00B1XZS820,ANGLO AMERICAN,549300S9XF92D1X8ME43,None,549300S9XF92D1X8ME43,"126,486.95","974,694.01"


In [90]:
non_il_fossil_holdings_by_issuer = non_il_fossil_holdings_agg.groupby([
    "ParentCorpGroup", "ReportPeriodDate", 'issuer_num'
], dropna=False)
non_il_fossil_holdings_by_issuer_agg = non_il_fossil_holdings_by_issuer.agg(
    name=pd.NamedAgg(column="name", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="fossil_sum", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="quantity_sum", aggfunc="sum")
).reset_index()

In [91]:
non_il_fossil_holdings_by_issuer_agg["fossil_sum"].sum()

3528104.9984420817

### add Israeli and non-Israeli holdings

In [92]:
fossil_holdings_by_issuer_agg = pd.concat([il_fossil_holdings_by_issuer_agg, non_il_fossil_holdings_by_issuer_agg])
# clean holding name
fossil_holdings_by_issuer_agg["clean_name"] = fossil_holdings_by_issuer_agg["name"].apply(clean_company)
fossil_holdings_by_issuer_agg["fossil_sum"].sum()

12357277.259338392

In [93]:
# group by issuer_num
fossil_holdings_by_issue_grp = fossil_holdings_by_issuer_agg.groupby([
    "ParentCorpGroup", "ReportPeriodDate", 'issuer_num'
], dropna=False)
fossil_holdings_by_issuer_agg = fossil_holdings_by_issue_grp.agg(
    name=pd.NamedAgg(column="clean_name", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="fossil_sum", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="quantity_sum", aggfunc="sum")
).reset_index()

# group by holding name (clean)
fossil_holdings_by_issue_grp = fossil_holdings_by_issuer_agg.groupby([
    "ParentCorpGroup", "ReportPeriodDate", 'name'
], dropna=False)
fossil_holdings_by_issuer_agg = fossil_holdings_by_issue_grp.agg(
    issuer_num=pd.NamedAgg(column="issuer_num", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="fossil_sum", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="quantity_sum", aggfunc="sum")
).reset_index()


In [94]:
fossil_holdings_by_issuer_agg

,ParentCorpGroup,ReportPeriodDate,name,issuer_num,fossil_sum,quantity_sum
0,מגדל,2021-06-30,ANGLO AMERICAN,549300S9XF92D1X8ME43,"126,486.95","974,694.01"
1,מגדל,2021-06-30,BPLN,549300CRVT18MXX0AG93,"131,935.78","37,000,000.00"
2,מגדל,2021-06-30,CQP,5493005UEC8AZ34LDV29,"113,377.14","32,100,000.00"
3,מגדל,2021-06-30,DELEK AVNER TAMAR,514914001,"218,168.52","66,528,061.98"
4,מגדל,2021-06-30,EDF PREP,549300X3UK4GG3FNMO06,"208,024.54","43,700,000.01"
5,מגדל,2021-06-30,ENBCN,98TPTUM4IVMFCZBCUR27,"78,426.28","21,509,999.97"
6,מגדל,2021-06-30,ENBRIDGE,None,"544,409.33","148,113,262.05"
7,מגדל,2021-06-30,ENERGEAN OIL GAS,549300RVMKU0CYUZBB05,"128,002.76","3,968,292.99"
8,מגדל,2021-06-30,GLENCORE,2138002658CPO9NBH955,"106,947.17","7,649,999.96"
9,מגדל,2021-06-30,MARATHON OIL,1FRVQX2CRLGC1XLP5727,10.92,246.00


In [95]:
# divide to quarters (take only first 2)
report_periods = np.sort(fossil_holdings_by_issuer_agg["ReportPeriodDate"].unique())
prev_q = fossil_holdings_by_issuer_agg[fossil_holdings_by_issuer_agg["ReportPeriodDate"] == report_periods[0]]
curr_q = fossil_holdings_by_issuer_agg[fossil_holdings_by_issuer_agg["ReportPeriodDate"] == report_periods[1]]

In [96]:
cols = ["name", "issuer_num", "fossil_sum", "quantity_sum"]
comparison = prev_q[cols].merge(curr_q[cols],
                                on="issuer_num",
                                how="outer",
                                suffixes=["_prev_q", "_curr_q"]
                               )
comparison["name_prev_q"] = comparison["name_prev_q"].fillna(comparison["name_curr_q"])
comparison = comparison.drop("name_curr_q", axis=1).rename({"name_prev_q":"name"}, axis=1)
comparison = comparison.fillna(0).sort_values("name")
comparison["fossil_quantity_diff"] = comparison["quantity_sum_curr_q"] - comparison["quantity_sum_prev_q"]
comparison["fossil_sum_diff"] = comparison["fossil_sum_curr_q"] - comparison["fossil_sum_prev_q"]

In [97]:
company_quarter = company_for_comparison[0] + " - " + quarters_for_comparison[0]
comparison.to_csv("analysis/fossil holdings comparison - " + company_quarter + ".csv", index=False)

# Change fossil classification for selected holdings, analyze results

In [17]:
# fix data in all_holdings_cls - wrong data for סופרגז
# all_holdings_cls[all_holdings_cls["שם המנפיק/שם נייר ערך"] == 'סופרגז']

In [49]:
all_holdings_cls.loc[all_holdings_cls["שם המנפיק/שם נייר ערך"] == 'סופרגז',
                    'מספר ני"ע'] = '1166917'

all_holdings_cls.loc[all_holdings_cls["שם המנפיק/שם נייר ערך"] == 'סופרגז',
                     'ISIN'] = 'IL0011669178'

In [50]:
all_holdings_cls["clean_name"] = all_holdings_cls["שם המנפיק/שם נייר ערך"].apply(clean_company)

In [68]:
selected_holdings_names = ["כיל", "ICL", 'איי סי אל', 'איי.סי.אל', 'אייסיאל', 'ISRAEL CHEMICAL', 'כימיקלים לישראל', 'כי"ל',
                           "רימון",
                           "סופרגז", "סופר גז",
                           "OPC", 'אופיסי', 'או. פי. סי', 'או פי סי',
                           'חברה לישראל', 'החברה לישראל', 'חברהלישראל','חברה ישראל']
name_mask = all_holdings_cls["clean_name"].str.upper().str.startswith(tuple(selected_holdings_names))
all_holdings_cls.loc[name_mask].groupby(['מספר מנפיק'], dropna=False).agg(
    name=pd.NamedAgg(column="clean_name", aggfunc="first")
)

,name
מספר מנפיק,
1395,רימון מניה לא סחירה
1486,סופרגז
1682,אופיסי
1804,סופרגז
1928,רימון
281,אייסיאל
28327,OPC POWER VENTURES LP
576,חברה לישראל
NaN,סופרגז


In [69]:
def holdings_col_mask_by_another_mask(holdings, col_name, mask):
    selected_col_values = holdings.loc[mask, col_name].dropna().unique()
    col_mask = holdings[col_name].isin(selected_col_values)
    print("matching holdings by {}: {}".format(col_name, col_mask.sum()))
    return col_mask

In [70]:
# by issuer num
issuer_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "מספר מנפיק", name_mask)
ISIN_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "ISIN", name_mask)
LEI_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "LEI", name_mask)
il_corp_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "מספר תאגיד", name_mask)
il_sec_num_mask = holdings_col_mask_by_another_mask(all_holdings_cls, 'מספר ני"ע', name_mask)

matching holdings by מספר מנפיק: 3216
matching holdings by ISIN: 2784
matching holdings by LEI: 0
matching holdings by מספר תאגיד: 3317
matching holdings by מספר ני"ע: 3104


In [71]:
all_ids_mask = name_mask | issuer_mask | ISIN_mask | LEI_mask | il_corp_mask | il_sec_num_mask
no_name_match = all_holdings_cls[all_ids_mask & (~name_mask)]
print("selected holdings with no name match: {}".format(no_name_match.shape[0]))

selected holdings with no name match: 234


In [74]:
no_name_match.sort_values("is_fossil", ascending=False)

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI,clean_name
303258,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.04,"3,604.36",0.01,0.00,1824549,"אג""ח קונצרני",TASE,NaN,2.39,"3,475,752.20",103.70,0.00,0.01,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,27/05/2020,2020 רבעון 1,1.00,"3,604.36",IL0057602364,520028010,NaN,חלישראל אג
516755,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.0114,"2,119.54",0.01,0.00,2150933,"אג""ח קונצרני",TASE,NaN,1.68,"1,974,053.22",107.37,0.00,0.00,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0,nan,nan,02/12/2021,2021 רבעון 3,1.00,"2,119.54",IL0057602364,520028010,NaN,חלישראל אג
260646,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.04,0.01,"5,154.03",0.02,0.00,2014144,"אג""ח קונצרני",TASE,NaN,2.03,"4,815,945.98",107.02,0.00,0.01,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,24/03/2021,2020 רבעון 4,1.00,"5,154.03",IL0057602364,520028010,NaN,חלישראל אג
149389,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.03,"3,218.08",0.01,0.00,1879029,"אג""ח קונצרני",TASE,NaN,2.47,"3,089,557.51",104.16,0.00,0.01,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,30/08/2020,2020 רבעון 2,1.00,"3,218.08",IL0057602364,520028010,NaN,חלישראל אג
203528,ISRAE 4.5 12/24,245217,281,ilBBB-,S&P מעלות,דולר אמריקאי,0.04,0.02,"3,658.36",0.00,0.00,1948374,"אג""ח קונצרני",TASE,NaN,3.82,"960,000.00",110.75,0.00,0.00,NaN,Materials,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"מיטב דש גמל ופנסיה בע""מ",I_512065202,0.0,nan,nan,06/12/2020,2020 רבעון 3,1.00,"3,658.36",None,520027830,NaN,ISRAE
306998,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.02,"4,758.65",0.02,0.00,1943203,"אג""ח קונצרני",TASE,NaN,2.24,"4,415,557.51",107.77,0.00,0.01,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,26/11/2020,2020 רבעון 3,1.00,"4,758.65",IL0057602364,520028010,NaN,חלישראל אג
470334,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.02,"2,600.16",0.01,0.00,2051177,"אג""ח קונצרני",TASE,NaN,1.79,"2,429,603.98",107.02,0.00,0.00,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,31/05/2021,2021 רבעון 1,1.00,"2,600.16",IL0057602364,520028010,NaN,חלישראל אג
468511,ISRAE 4.5 12/24,None,281,BBB-,S&P,דולר אמריקאי,0.04,0.01,"3,484.24",0.00,0.00,2016501,"אג""ח קונצרני",אחר,NaN,3.65,"960,000.00",112.89,0.00,0.00,בלומברג,Materials,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"מיטב דש גמל ופנסיה בע""מ",I_512065202,0.0,nan,nan,31/03/2021,2020 רבעון 4,1.00,"3,484.24",IL0028102734,520027830,NaN,ISRAE
456335,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.01,"2,115.20",0.01,0.00,2104058,"אג""ח קונצרני",TASE,NaN,1.93,"1,974,053.22",107.15,0.00,0.00,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,02/09/2021,2021 רבעון 2,1.00,"2,115.20",IL0057602364,520028010,NaN,חלישראל אג
437836,ISHARES S&P GBL CLEAN ENERGY,None,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,16.80,0.00,0.00,1949813,קרנות סל,NYSE,NaN,nan,264.00,"1,849.00",0.00,0.00,NaN,מניות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,07/12/2020,

In [73]:
all_holdings_cls.loc[all_ids_mask, "is_fossil"].value_counts(dropna=False)

1.00    3193
nan      304
0.00      32
Name: is_fossil, dtype: int64

In [103]:
# change classification
all_holdings_cls.loc[all_ids_mask, "is_fossil"] = 0
all_holdings_cls["שווי פוסילי"] = all_holdings_cls["שווי"] * all_holdings_cls["is_fossil"]